In [ ]:
sc

### UDF

In [ ]:
players =  spark.read.format("csv")\
                .option("header","true")\
                .load("file:///home/icpl12900/Desktop/assignments/Spark_data/player.csv")
        

In [ ]:
player_attribute = spark.read.format("csv")\
                .option("header","true")\
                .load("file:///home/icpl12900/Desktop/assignments/Spark_data/player_attributes.csv")

In [ ]:
player_attribute.printSchema()

In [ ]:
players.printSchema()

In [ ]:
from pyspark.sql.functions import udf

In [ ]:
udf

In [ ]:
year_extract_udf = udf(lambda date : date.split('-')[0])

In [ ]:
new_player_attribute = player_attribute.withColumn('year' , year_extract_udf(player_attribute.date))

In [ ]:
new_player_attribute.printSchema()

In [ ]:
pa_2016 = new_player_attribute.filter(new_player_attribute.year == 2016)

In [ ]:
pa_2016.show(2)

In [ ]:
pa_striker_2016 = pa_2016.groupBy('player_api_id').\
                        agg({"finishing" : 'avg',
                            'shot_power':'avg',
                            'acceleration':'avg'})

In [ ]:
pa_striker_2016.printSchema()

In [ ]:
pa_striker_2016=pa_striker_2016.withColumnRenamed('avg(finishing)','finishing')\
              .withColumnRenamed('avg(acceleration)','acceleration')\
              .withColumnRenamed('avg(shot_power)','shot_power')

In [ ]:
pa_striker_2016.printSchema()

In [ ]:
finishing = 1
acceleration = 2
shot_power = 1
total = finishing + acceleration + shot_power

In [ ]:
total

In [ ]:
striker = pa_striker_2016.withColumn("striker_grade" , (pa_striker_2016.finishing * finishing +\
                                                       pa_striker_2016.acceleration * acceleration +\
                                                       pa_striker_2016.shot_power * shot_power)/total)

In [ ]:
striker.printSchema()

In [ ]:
striker = striker.sort(striker.striker_grade.desc())

In [ ]:
striker.show(5)

In [ ]:
striker=striker.drop('finishing','acceleration','shot_power')

In [ ]:
striker.show(2)

In [ ]:
players.printSchema()

In [ ]:
best_striker = striker.join(players,players.player_api_id == striker.player_api_id )

In [ ]:
best_striker.show(2)

In [ ]:
best_striker = striker.join(players,['player_api_id'] )

In [ ]:
best_striker.show(2)

### BROADCAST VARIABLES

WHEN DATASETS IS HUGE THEN JOIN OPERATIONS ON THAT DATASET IS VERY HEAVY OPERATION. BCOZ DATASETS ARE DISTRIBUTE ACROSS NODES IN CLUSTER. SO USE BROADCAST IN WHICH ONE DATASET WHICH IS SMALL IS COPY TO WORKER NODE AND JOIN PRFORM OVER THERE.

In [ ]:
from pyspark.sql.functions import broadcast

In [ ]:
striker_details = players.select('player_api_id' ,'player_name').\
                            join(
                                    broadcast(striker),
                                    ['player_api_id'],
                                    'inner' )

In [ ]:
striker_details.show(2)

### ACCUMULATOR

#### HIGH HEADING ACCURACY DIVIDED BY BUCKET

In [ ]:
players_heading_acc = player_attribute.select('player_api_id','heading_accuracy').\
                                       join(
                                            broadcast(players),
                                                 ['player_api_id'],
                                                'inner' 
                                            )

In [ ]:
players_heading_acc.printSchema()

#### HEIGHT

In [ ]:
short = spark.sparkContext.accumulator(0)
medium = spark.sparkContext.accumulator(0)
tall = spark.sparkContext.accumulator(0)

In [ ]:
def count_player_height(row):
    height = float(row.height)
    if (height<=175):
        short.add(1)
    elif (height <= 195):
        medium.add(1)
    else:
        tall.add(1)
    

In [ ]:
players_heading_acc.foreach(lambda x : count_player_height(x))

In [ ]:
all_players = [short.value,
              medium.value,
              tall.value]
all_players

### Heading_accuracy

In [ ]:
short_ha = spark.sparkContext.accumulator(0)
medium_ha = spark.sparkContext.accumulator(0)
tall_ha = spark.sparkContext.accumulator(0)

In [ ]:
def count_player_headingAccuracy(row):
    head = row.heading_accuracy
    if(head>=75 and head<=175):
        short_ha.add(1)
    elif (head <= 195):
        medium_ha.add(1)
    else:
        tall_ha.add(1)

In [ ]:
players_heading_acc.foreach(lambda x : count_player_headingAccuracy(x))

In [ ]:
all_players_heading = [short_ha.value,
                      medium_ha.value,
                      tall_ha.value]
all_players_heading

In [ ]:
percentage_value = [(short_ha.value/short.value) *100,
                      (medium_ha.value/medium.value) *100,
                      (tall_ha.value/tall.value) *100]

percentage_value

## STORE DATAFRAME TO FILE

In [ ]:
pa_2016.columns

In [ ]:
pa_2016.select( 'player_api_id','date')\
        .coalesce(1)\
        .write\
        .option("header","true")\
        .csv("file:///home/icpl12900/Desktop/assignments/Spark_data/player_date.csv")
        
#coalesce(1) --- to repartitioned DataFrame to single partition, to write into single file. here we want 
#one partition  so use 1 as argument
#player_date.csv it is directiory

## USE CUSTOM ACCUMULATOR

In [ ]:
from pyspark.accumulators import AccumulatorParam

In [ ]:
class VectorAccumulator(AccumulatorParam):
    def zero(self,value):
        return [0.0] * len(value)
    def addInPlace(self,v1,v2):
        for i in range(len(v1)):
            v1[i] += v2[i]
        return v1

In [ ]:
vector_acc = sc.accumulator([10.0,20.0],VectorAccumulator())
vector_acc.value

## BASIC SQL OPERATIONS ON SPARK

In [ ]:
pa_2016

In [ ]:
pa_2016.createOrReplaceTempView('table_2016')


##### It create temporary view for dataFrame pa_2016 and it is temporary . It create only for this session. Once session is closed view also disappear
#### It Register DataFrame as Table For Per-Session

In [ ]:
all_data = sqlContext.sql("select * from table_2016")

In [ ]:
all_data.show(1)

In [ ]:
sqlContext.sql('select id from table_2016 where id == 1').show()

##### To make avaliable Global table to all SPARK SESSION  use "createGlobalTempView"
#### And to access this global table use  "global_temp.tableName"

In [ ]:
pa_2016.createGlobalTempView('table_2016')

In [ ]:
sqlContext.sql("select * from global_temp.table_2016").show(1)